In [1]:
import pandas as pd
import yaml
from pathlib import Path
from costometer.utils import AnalysisObject
import pingouin as pg

/Users/valkyrie/git/planning-depth-differences/venv/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
/Users/valkyrie/git/planning-depth-differences/venv/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


# Prepare data for looking at first 10 vs last 10 trials in test 

In [2]:
with open(Path("../../../data/inputs/yamls/experiments/methods_main.yaml"), "rb") as f:
    data = yaml.safe_load(f)

data['trials_per_block'] = {'test_begin':10, 'test_end':10}
with open(Path(f"../../../data/inputs/yamls/experiments/methods_main_transfer.yaml"), "w") as f:
    yaml.dump(data, f)

In [3]:
mouselab_data = pd.read_csv(Path("../../../data/processed/methods_main/mouselab-mdp.csv"), index_col=0)

ranges_to_extract = {'none':"range(0,20)", 'test_begin': "range(20,30)", "test_end": "range(30,40)"}
    
# TODO: refactor out
def expand_range_dictionary(input_dictionary):
    trial_to_block = {}
    for block, trial_range in input_dictionary.items():
        if isinstance(trial_range, str):
            for trial_index in eval(trial_range):
                trial_to_block[trial_index] = block
        else:
            trial_to_block[block] = expand_range_dictionary(trial_range)
    return trial_to_block

trial_to_block = expand_range_dictionary(ranges_to_extract)

mouselab_data["block"] = mouselab_data.apply(lambda row: trial_to_block[row["run"]][row["trial_index"]] if row["run"] in trial_to_block else trial_to_block[row["trial_index"]], axis=1)

mouselab_data = mouselab_data[mouselab_data["block"].isin(["test_begin", "test_end"])]

Path(f"../../../data/processed/methods_main_transfer/").mkdir(exist_ok=True, parents=True)
mouselab_data.to_csv(Path(f"../../../data/processed/methods_main_transfer/mouselab-mdp.csv"), index=False)

In [4]:
mouselab_data.groupby(["pid", "block"]).count()

success  trial_type  time_elapsed  internal_node_id  \
pid  block                                                             
1723 test_begin        0          10            10                10   
     test_end          0          10            10                10   
1731 test_begin        0          10            10                10   
     test_end          0          10            10                10   
1737 test_begin        0          10            10                10   
...                  ...         ...           ...               ...   
2372 test_end          0          10            10                10   
2373 test_begin        0          10            10                10   
     test_end          0          10            10                10   
2374 test_begin        0          10            10                10   
     test_end          0          10            10                10   

                 view_history  rt  responses  correct  revealed_states  \
pid  block                                                               
1723 test_begin             0  10          0        0               10   
     test_end               0  10          0        0               10   
1731 test_begin             0  10          0        0               10   
     test_end               0  10          0        0               10   
1737 test_begin             0  10          0        0               10   
...                       ...  ..        ...      ...              ...   
2372 test_end               0  10          0        0               10   
2373 test_begin             0  10          0        0               10   
     test_end               0  10          0        0               10   
2374 test_begin             0  10          0        0               10   
     test_end               0  10          0        0               10   

                 num_clicks_accrued  ...  variance  branching  final_bonus  \
pid  block                           ...                                     
1723 test_begin                  10  ...        10         10           10   
     test_end                    10  ...        10         10           10   
1731 test_begin                  10  ...        10         10           10   
     test_end                    10  ...        10         10           10   
1737 test_begin                  10  ...        10         10           10   
...                             ...  ...       ...        ...          ...   
2372 test_end                    10  ...        10         10           10   
2373 test_begin                  10  ...        10         10           10   
     test_end                    10  ...        10         10           10   
2374 test_begin                  10  ...        10         10           10   
     test_end                    10  ...        10         10           10   

                 displayed_bonus  time_diff  num_nodes  num_early  num_middle  \
pid  block                                                                      
1723 test_begin               10         10         10         10          10   
     test_end                 10         10         10         10          10   
1731 test_begin               10         10         10         10          10   
     test_end                 10         10         10         10          10   
1737 test_begin               10         10         10         10          10   
...                          ...        ...        ...        ...         ...   
2372 test_end                 10         10         10         10          10   
2373 test_begin               10         10         10         10          10   
     test_end                 10         10         10         10          10   
2374 test_begin               10         10         10         10          10   
     test_end                 10         10         10         10          10   

                 num_late  num_clicks  
pid  blo

In [5]:
for file_idx in range(10):
    print(f"condor_submit_bid 1 submission_scripts/MPI-IS/04_Infer_Params.sub experiment=methods_main_transfer cost_function=back_dist_depth_eff_forw param_file=params_full_five_modified{file_idx}")

condor_submit_bid 1 submission_scripts/MPI-IS/04_Infer_Params.sub experiment=methods_main_transfer cost_function=back_dist_depth_eff_forw param_file=params_full_five_modified0
condor_submit_bid 1 submission_scripts/MPI-IS/04_Infer_Params.sub experiment=methods_main_transfer cost_function=back_dist_depth_eff_forw param_file=params_full_five_modified1
condor_submit_bid 1 submission_scripts/MPI-IS/04_Infer_Params.sub experiment=methods_main_transfer cost_function=back_dist_depth_eff_forw param_file=params_full_five_modified2
condor_submit_bid 1 submission_scripts/MPI-IS/04_Infer_Params.sub experiment=methods_main_transfer cost_function=back_dist_depth_eff_forw param_file=params_full_five_modified3
condor_submit_bid 1 submission_scripts/MPI-IS/04_Infer_Params.sub experiment=methods_main_transfer cost_function=back_dist_depth_eff_forw param_file=params_full_five_modified4
condor_submit_bid 1 submission_scripts/MPI-IS/04_Infer_Params.sub experiment=methods_main_transfer cost_function=back_di

condor_submit_bid 1 submission_scripts/MPI-IS/05_Combine_Human.sub experiment=methods_main_transfer cost_function=back_dist_depth_eff_forw

condor_submit_bid 1 submission_scripts/MPI-IS/M_01_Get_MAP_File.sub experiment=methods_main_transfer cost_function=back_dist_depth_eff_forw

# Look at training vs test for now

In [6]:
analysis_obj = AnalysisObject(
    "MainExperiment",
    irl_path=Path("../../..").resolve(),
    experiment_subdirectory="methods/static",
)

In [7]:
training_fit = analysis_obj.query_optimization_data(group=False, prior="expon", block=["training"], include_null=True)
training_fit.groupby(["Model Name"]).sum()["bic"].sort_values()

/Users/valkyrie/git/costometer/costometer/utils/analysis_utils.py:702: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  subset.groupby(["Model Name", "cost_function"]).sum()["bic"].reset_index()
/var/folders/y7/qnlh5f453nxbc7w72qw0lw340000gn/T/ipykernel_17939/2867809176.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  training_fit.groupby(["Model Name"]).sum()["bic"].sort_values()


Model Name
Distance, Effort, Depth and Forward Search Bonus              54782.219530
Distance, Depth and Forward Search Bonus                      55116.873211
All five                                                      55339.071538
Distance, Depth, Backward and Forward Search Bonus            55653.136644
Distance and Depth Costs                                      57682.519700
Distance, Depth and Effort Costs                              57698.088337
Effort, Depth and Forward Search Bonus                        57719.917203
Distance, Effort and Forward Search Bonus                     57768.735867
Distance and Forward Search Bonus                             58019.958622
Effort, Depth, Backward and Forward Search Bonus              58137.744637
Distance, Depth and Backward Search Bonus                     58242.233464
Distance, Depth and Effort Costs and Backward Search Bonus    58290.278636
Distance, Effort, Backward and Forward Search Bonus           58332.690339
Depth and Forw

In [8]:
test_fit = analysis_obj.query_optimization_data(group=False, prior="expon", block=["test"], include_null=True)
test_fit.groupby(["Model Name"]).sum()["bic"].sort_values()

/Users/valkyrie/git/costometer/costometer/utils/analysis_utils.py:702: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  subset.groupby(["Model Name", "cost_function"]).sum()["bic"].reset_index()
/var/folders/y7/qnlh5f453nxbc7w72qw0lw340000gn/T/ipykernel_17939/4063575225.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  test_fit.groupby(["Model Name"]).sum()["bic"].sort_values()


Model Name
Distance, Effort, Depth and Forward Search Bonus              40490.817744
Distance, Depth and Forward Search Bonus                      40812.279744
All five                                                      41043.623143
Distance, Depth, Backward and Forward Search Bonus            41486.966667
Distance, Depth and Effort Costs                              41960.629832
Distance and Depth Costs                                      41971.822614
Distance, Effort and Forward Search Bonus                     42486.270816
Distance, Depth and Effort Costs and Backward Search Bonus    42600.549378
Distance, Depth and Backward Search Bonus                     42672.650610
Effort, Depth and Forward Search Bonus                        43015.229860
Distance and Effort Costs                                     43085.029931
Distance, Effort, Backward and Forward Search Bonus           43111.330899
Effort, Depth, Backward and Forward Search Bonus              43463.985457
Distance and F

In [9]:
model = "Distance, Effort, Depth and Forward Search Bonus"
cost_functions = ['forw_added_cost', 'given_cost', 'distance_multiplier', 'depth_cost_weight', 'temp']

for cost_function in cost_functions:
    print(cost_function)
    print(pg.wilcoxon(test_fit[test_fit["Model Name"]==model].sort_values("trace_pid")[cost_function],\
    training_fit[training_fit["Model Name"]==model].sort_values("trace_pid")[cost_function]))

forw_added_cost
          W-val alternative         p-val       RBC      CLES
Wilcoxon  115.0   two-sided  7.992375e-10  0.885913  0.646869
given_cost
          W-val alternative     p-val       RBC      CLES
Wilcoxon  483.5   two-sided  0.000194  0.535096  0.570176
distance_multiplier
          W-val alternative         p-val       RBC      CLES
Wilcoxon  481.0   two-sided  1.623434e-10 -0.770186  0.287792
depth_cost_weight
          W-val alternative     p-val      RBC      CLES
Wilcoxon  841.5   two-sided  0.000009 -0.55012  0.392435
temp
          W-val alternative         p-val       RBC      CLES
Wilcoxon  295.5   two-sided  2.652958e-14 -0.880606  0.228971


In [10]:
model = "All five"
cost_functions = ["back_added_cost",'forw_added_cost', 'given_cost', 'distance_multiplier', 'depth_cost_weight', 'temp']

for cost_function in cost_functions:
    print(cost_function)
    print(pg.wilcoxon(test_fit[test_fit["Model Name"]==model].sort_values("trace_pid")[cost_function],\
    training_fit[training_fit["Model Name"]==model].sort_values("trace_pid")[cost_function]))

back_added_cost
          W-val alternative     p-val       RBC      CLES
Wilcoxon  436.5   two-sided  0.001133  0.489772  0.541017
forw_added_cost
          W-val alternative         p-val      RBC      CLES
Wilcoxon  168.5   two-sided  2.819145e-09  0.84289  0.648985
given_cost
          W-val alternative     p-val       RBC      CLES
Wilcoxon  323.0   two-sided  0.000036  0.622443  0.574308
distance_multiplier
          W-val alternative         p-val       RBC     CLES
Wilcoxon  573.5   two-sided  1.686978e-09 -0.725991  0.29478
depth_cost_weight
          W-val alternative     p-val      RBC      CLES
Wilcoxon  756.5   two-sided  0.000004 -0.57619  0.391125
temp
          W-val alternative         p-val       RBC      CLES
Wilcoxon  325.0   two-sided  3.682665e-14 -0.871287  0.232061
